## Model IO 

### Few Shot Prompt Template 

- prompt template를 사용하는 이유는 디스크에 저장하고 로드할 수 있기 때문임.

- Fewshot은 기본적으로 모델에게 예제를 준다는 뜻과 같다. 더 나은 대답을 할 수 있도록할 수 있도록 예제를 줄 수 있다.
- 구체적인 디렉션을 줘서 prompt로 주는 것보다 예제를 보여주는 것이 훨씬 좋다. 

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?", # suffix는 형식화 된 모든 예제 마지막에 나오는 내용임.
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Turkey"})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

#### FewShotChatMessagePromptTemplate

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

### LengthBasedExamplesSelector


- 예제들을 형식화 할 수 있고 예제의 양을 확인할 수 있다. 설정해 놓은 세팅값에 따라 프롬프트에 알맞은 예제를 골라줄 수 있음. 


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

### Serialization and Composition

Serialization (직렬화)
Serialization이란, 객체의 상태를 저장하거나 네트워크를 통해 전송할 수 있는 형식으로 변환하는 과정을 말합니다. 이는 모델의 입력 데이터 또는 출력 데이터를 적절한 형식으로 변환하는 것을 포함할 수 있습니다. LangChain에서는 자연어 처리 모델과 같은 AI 모델의 입력과 출력을 처리하기 위해 Serialization을 사용할 수 있습니다. 예를 들어, 텍스트 데이터를 모델이 이해할 수 있는 숫자 형식으로 변환하거나, 모델의 출력을 사용자가 이해할 수 있는 텍스트로 변환하는 과정을 포함할 수 있습니다.

Composition (구성)
Composition은 여러 AI 모델 또는 기능을 결합하여 복잡한 작업을 수행하는 방법을 말합니다. 예를 들어, 하나의 작업을 수행하기 위해 텍스트 생성 모델, 분류 모델, 요약 모델 등 여러 모델을 순차적이거나 병렬적으로 연결할 수 있습니다. LangChain에서는 이러한 Composition을 통해 개별 모델의 한계를 극복하고, 더 풍부하고 다양한 AI 기반 응용 프로그램을 개발할 수 있습니다. 예를 들어, 사용자 질문을 입력으로 받아 이를 분석하는 모델로 전달하기 전에 특정 형식으로 변환하는 직렬화 과정을 거친 후, 해당 분석 결과를 바탕으로 적절한 답변을 생성하는 텍스트 생성 모델에 입력할 수 있습니다. 이 과정에서 필요에 따라 추가적인 정보 검색이나 처리 단계를 구성할 수도 있습니다.


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
        You are a role playing assistant.
        And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
        This is an example of how you talk:

        Human: {example_question}
        You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
        Start now!

        Human: {question}
        You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrrg! Me favorite food be a hearty stew made with fresh fish and vegetables, matey! Nothing beats a good meal after a long day of plundering the high seas! Arrr!

AIMessageChunk(content='Arrrrg! Me favorite food be a hearty stew made with fresh fish and vegetables, matey! Nothing beats a good meal after a long day of plundering the high seas! Arrr!')